In [5]:
# Importing Libraries

import os 
from google.cloud import bigquery

In [6]:
# Setting up environment, creating client

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/Sosa/Downloads/ironhack_service_account_big_query.json"

client = bigquery.Client()

# Step 1: Calculate the royalties of each sales for each author
Write a SELECT query to obtain the following output:

Title ID

Author ID

Royalty of each sale for each author

The formula is:
  sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100
  
Note that titles.royalty and titleauthor.royaltyper are divided by 100 respectively because they are percentage numbers instead of floats.

In the output of this step, each title may appear more than once for each author. This is because a title can have more than one sales.

In [8]:
qry= '''
SELECT

    title_id AS Title_ID,
    au_id AS Author_ID,
    (titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) AS sales_royalty
    


FROM
    `ironhack-data-analytics-265219.publications.titleauthor` titleauthor

JOIN
    `ironhack-data-analytics-265219.publications.authors` authors
ON
    titleauthor.au_id = authors.au_id
    
JOIN
    `ironhack-data-analytics-265219.publications.titles` titles
ON
    titles.title_id = titleauthor.title_id

JOIN
    `ironhack-data-analytics-265219.publications.sales` sales
ON
    sales.title_id = titleauthor.title_id
    
ORDER BY
    sales_royalty DESC
'''

query_job = client.query(query= qry)
query_job.to_dataframe()

BadRequest: 400 Column name title_id is ambiguous at [4:5]

(job ID: 12daae35-c3e7-4ef2-9392-54fc77213eb9)

                                     -----Query Job SQL Follows-----                                      

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:SELECT
   3:
   4:    title_id AS Title_ID,
   5:    au_id AS Author_ID,
   6:    (titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) AS sales_royalty
   7:    
   8:
   9:
  10:FROM
  11:    `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
  12:
  13:JOIN
  14:    `ironhack-data-analytics-265219.publications.authors` authors
  15:ON
  16:    titleauthor.au_id = authors.au_id
  17:    
  18:JOIN
  19:    `ironhack-data-analytics-265219.publications.titles` titles
  20:ON
  21:    titles.title_id = titleauthor.title_id
  22:
  23:JOIN
  24:    `ironhack-data-analytics-265219.publications.sales` sales
  25:ON
  

# Step 2: Aggregate the total royalties for each title for each author
Using the output from Step 1, write a query to obtain the following output:

Title ID

Author ID

Aggregated royalties of each title for each author

Hint: use the SUM subquery and group by both au_id and title_id
In the output of this step, each title should appear only once for each author.

In [ ]:
qry= '''

SELECT

    title_id AS Title_ID,
    au_id AS author_id,
    au_lname AS Last_Name,
    au_fname AS First_Name,
    advance AS Advance,
    SUM(royalties) AS royalties, 
    
FROM 

    (SELECT
        titles.title_id,
        titles.price,
        titles.advance,
        titles.royalty,
        sales.qty,
        authors.au_id,
        au_lname,
        au_fname,
        titleauthor.royaltyper,
        (titles.price * sales.qty * titles.royalty/100 * titleauthor.royaltyper / 100) AS royalties
    FROM 
        `ironhack-data-analytics-265219.publications.titles` titles
    JOIN
        `ironhack-data-analytics-265219.publications.sales` sales
    ON
        sales.title_id = titles.title_id
    JOIN
        `ironhack-data-analytics-265219.publications.titleauthor` titleauthor 
    ON
        titleauthor.title_id = sales.title_id
    JOIN
        `ironhack-data-analytics-265219.publications.authors` authors 
    ON
        authors.au_id = titleauthor.au_id) subquery
        
GROUP BY
    1,2,3,4,5
'''

query_job = client.query(query= qry)
query_job.to_dataframe()

# Step 3: Calculate the total profits of each author
Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the output from Step 2, write a query to obtain the following output:

Author ID

Profits of each author by aggregating the advance and total royalties of each title

Sort the output based on a total profits from high to low, and limit the number of rows to 3.

In [ ]:
qry=('''

WITH royalties AS(

    SELECT
    
        authors.au_id AS Author_id,
        authors.au_lname AS Last_name,
        authors.au_fname AS First_Name,
        SUM(titles.advance * titleauthor.royaltyper / 100) AS Advance,
        SUM(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) AS royalties
        
    FROM 
    
        `ironhack-data-analytics-265219.publications.authors` authors
        
    JOIN
        `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
    ON
        titleauthor.au_id = authors.au_id
        
    JOIN
        `ironhack-data-analytics-265219.publications.titles` titles
    ON 
        titleauthor.title_id = titles.title_id
        
    JOIN
        `ironhack-data-analytics-265219.publications.sales` sales
    ON
        titleauthor.title_id = sales.title_id
        
    GROUP BY
        1,2,3
    ORDER BY
        authors.au_id ASC
    )

SELECT 

    royalties.author_id AS Author_id,
    royalties.au_lname AS Last_name,
    royalties.au_fname AS First_name,
    (royalties.advance + royalties.royalties) AS Profit
    
FROM 
    royalties
    
GROUP BY
    1,2,3,4
    
ORDER BY
    total_profit DESC
    
LIMIT 3
''')

query_job = client.query(query= qry)
query_job.to_dataframe()

# Challenge 2
Elevating from your solution in Challenge 1 , create a table named most_profiting_authors to hold the data about the most profiting authors. The table should have 2 columns:

au_id - Author ID

profits - The profits of the author aggregating the advances and royalties

Include your solution in solutions.ipynb.